In [58]:
import numpy as np
import torch
from math import e
Nr=4;Nt=4;M=128
channel = torch.from_numpy(np.load('noisechannel.npy',allow_pickle=True))
dlink = torch.from_numpy(np.load('directlink.npy',allow_pickle=True))
rlink = torch.from_numpy(np.load('receiverlink.npy',allow_pickle=True))
tlink = torch.from_numpy(np.load('transmitterlink.npy',allow_pickle=True))
ochannel = channel.view(18100,Nr**2,Nt**2)
dchannel = dlink.view(18100,Nr**2,Nt**2)
rchannel = rlink.view(18100,Nr**2,M)     
tchannel = tlink.view(M,Nt**2)

In [65]:
K =4
M=128
#transmit power covariance matrix
Pt = 20 # in watts
P_noise = 10**-14
P = (Pt/K*torch.eye(K))
Q = torch.sqrt(P)
Qinv = torch.linalg.inv(Q)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# phase inilization
phi = (1j*torch.ones(M,dtype=torch.cdouble)).to(device)
phi_inv = (1/phi).to(device)
phi = torch.diag(phi)
phi_inv = torch.diag(phi_inv)
rand_phi = ((torch.rand(M,dtype=torch.cdouble)+1j*torch.rand(M,dtype=torch.cdouble)-0.5-1j*0.5)*2).to(device)
rand_phi = rand_phi/abs(rand_phi).to(device)
rand_phi = torch.diag(rand_phi).to(device)
y0 = phi.flatten()
#random use index 
uindex = (np.random.permutation(len(ochannel))).reshape(-1,4) #random idex of dataset
N_labels = 1 # number of label to yield
T = 10000 #maximum interation
T2 = 500000 #maximum interation
epsilon = 0.001 #threshold
H1 = tchannel.to(device)
H2 = torch.zeros(size=(K,Nr**2,M),dtype=torch.cdouble).to(device)
H2pp = torch.zeros(size=(K,Nr**2,M),dtype=torch.cdouble).to(device)
H2p = torch.zeros((K*Nr**2,M)).to(device)
A = torch.zeros(size=(M,M),dtype=torch.cdouble).to(device)
traceyay = torch.zeros(size=(M,M),dtype=torch.cdouble).to(device)
deltayay = torch.zeros(M,M).to(device)
H0 = torch.zeros(size=(K,Nr**2,Nt**2), dtype=torch.cdouble).to(device)
yi = torch.zeros(M**2,dtype=torch.cdouble).to(device)

#H1 = 1.3e-5*(torch.randn(size=(M,Nt**2),dtype=torch.cdouble)+1j*torch.randn(size=(M,Nt**2),dtype=torch.cdouble)).to(device)

for t in range(N_labels):
    index = uindex[t,:]
    for i in range(len(index)):
        H0[i,:,:] = 0.0002*dchannel[index[i],:,:]
        H2[i,:,:] = (rchannel[index[i],:,:]) # K*Nr*M
        H2pp[i,:,:] = Qinv[i,i]*H2[i,:,:]
        H2p = H2pp.reshape(Nr**2*K,M) #NrK*M
        H2_inv = torch.linalg.pinv(H2p)# M * N_rK
        H1_inv = torch.linalg.pinv(H1) # Nt * M
        H22 = (H2_inv).conj().T # # N_rK * M 
        H1_inv = H1_inv.contiguous()
        H22 = H22.contiguous()
        caschannel = torch.kron(H22,H1_inv)
        A = (caschannel.conj().T) @ (caschannel)
    yay = (y0).conj().T@(A)@(y0)
    
print(torch.linalg.matrix_rank(caschannel))


tensor(192, device='cuda:0')


In [66]:
# SPF
lambda_max = torch.max(torch.real(torch.linalg.eigvals(A)))
print(lambda_max)
bigc = (lambda_max* torch.eye(M**2,M**2).to(device)-(A))
F = torch.zeros(M,dtype=torch.cdouble).to(device)
for tt in range(T2):
    D = bigc @ y0
    E = torch.diag(D.view(M,M))
    F = e**(1j*torch.angle(E))
    #print(torch.norm(F - torch.diag(y0.view(M,M))))
    if torch.norm(F - torch.diag(y0.view(M,M))) < epsilon :
        optimal_phase = F
        break
    else: y0 = torch.diag(F).flatten() 
print(tt)
print(optimal_phase)
optimal_P = torch.norm(torch.norm(H2@H1,dim=1),dim=1)/torch.sum(torch.norm(torch.norm(H2@H1,dim=1),dim=1))*Pt


tensor(1.5473e+19, device='cuda:0', dtype=torch.float64)
254707
tensor([-0.9063+0.4227j, -0.5827-0.8127j, -0.1625+0.9867j, -0.0163+0.9999j,
         0.7603+0.6496j, -0.8417-0.5400j, -0.9545+0.2982j, -0.9862-0.1655j,
         0.0871+0.9962j, -0.8808+0.4735j,  0.7183+0.6958j,  0.1797+0.9837j,
         0.0192+0.9998j, -0.5536+0.8328j,  0.9796+0.2009j, -0.0084+1.0000j,
         0.0131+0.9999j,  0.9563+0.2923j, -0.9990+0.0457j,  0.3570+0.9341j,
         0.9860-0.1670j, -0.5480+0.8365j,  0.4819+0.8762j,  0.7163+0.6978j,
        -0.9406+0.3394j,  0.8137+0.5814j,  0.2693+0.9631j, -0.9118-0.4106j,
         0.9999+0.0167j, -0.8294-0.5587j,  0.5978+0.8016j,  0.8539+0.5205j,
         0.0525+0.9986j,  0.3273-0.9449j,  0.5043+0.8635j, -0.9579+0.2872j,
        -0.0641+0.9979j,  0.0346+0.9994j,  0.5933-0.8050j,  0.5535+0.8329j,
        -0.6836+0.7299j,  0.9223+0.3864j, -0.3511-0.9363j,  0.6037+0.7972j,
        -0.4982+0.8671j,  0.6829+0.7305j,  0.8301-0.5577j,  0.3192+0.9477j,
         0.1541+0.9881j,

In [61]:
    #gradient based
    for ii in range(M):
        for jj in range(M):
            II = (ii-1)*M+ii
            JJ = (jj-1)*M+jj
            traceyay[ii,jj] = A[II,JJ]
            allelements = torch.mul(traceyay.flatten(),(torch.diag(phi)).repeat(M))
            upper = 1j*e**(1j*(phi)[ii,jj])*torch.sum(allelements[:(int(len(allelements)/2)-1)])
            lower = -1j*e**(-1j*(phi)[ii,jj])*torch.sum(allelements[(int(len(allelements)/2)-1):])
            deltayay[ii,jj] = (2*torch.real(upper + lower))
    
    q0 = deltayay.flatten().to(device)
    d0 = -q0.to(device)
    yt = phi_inv.flatten().to(device)
    for t1 in range(T):
        t1 = t1/1
        y1t = torch.mul(yt,d0).to(device)
        y2t = torch.mul(y1t,d0).to(device)
        z2 = torch.imag(yt.conj().T@A@y1t).to(device)
        z3 = (torch.real(yt.conj().T@A@y2t)-y1t.conj().T@A@y1t).to(device)
        ustep = (-z2/z3).to(device)
        y_next = torch.mul(torch.mul(yt, e**(1j*ustep*d0)),(torch.eye(M).flatten()).to(device))
        q_next = 2*torch.real(-1j*torch.mul(y_next.T, A@y_next))
        d_next = -q_next + ((q_next-q0).T @ q_next)/((torch.norm(q0))**2)*d0
        if q_next.T @ d_next >= 0:
            d_next = -q_next
        if torch.norm(y_next-yt) < epsilon :
            optimal_phi = y_next
            break
        
        yt = y_next
        d0 = d_next
        q0 = q_next
        #if torch.log10(torch.abs(yt.conj().T @ A @ yt)) <= Pt:
        optimal_P = torch.norm(torch.norm(H2@H1,dim=1),dim=1)/torch.sum(torch.norm(torch.norm(H2@H1,dim=1),dim=1))*Pt
            #break
        #else: optimal_P  = Pt/K
g_phase = torch.diag(optimal_phi.view(M,M))/abs(torch.diag(optimal_phi.view(M,M)))
print(t1)
print(g_phase)

31.0
tensor([ 9.2362e-01-3.8330e-01j, -9.9036e-01-1.3848e-01j,
        -9.7504e-01-2.2204e-01j, -9.2728e-01-3.7436e-01j,
         9.4278e-01-3.3341e-01j, -4.3033e-01-9.0267e-01j,
         9.6101e-01+2.7652e-01j,  6.8143e-01-7.3188e-01j,
         9.9909e-01-4.2612e-02j,  9.9315e-01+1.1687e-01j,
         9.7270e-01+2.3206e-01j,  6.6212e-01-7.4940e-01j,
         8.4835e-01+5.2943e-01j, -7.5838e-01-6.5181e-01j,
        -9.1761e-01+3.9748e-01j,  2.3549e-01-9.7188e-01j,
         8.3294e-01-5.5337e-01j, -9.2860e-01-3.7107e-01j,
         7.0536e-01-7.0885e-01j,  8.4578e-01-5.3353e-01j,
         7.4470e-02-9.9722e-01j, -8.8026e-01-4.7450e-01j,
         9.8712e-01-1.5997e-01j, -4.7759e-01-8.7858e-01j,
         8.2339e-01-5.6748e-01j, -4.5840e-01-8.8875e-01j,
        -9.8026e-01+1.9770e-01j,  9.6738e-01-2.5333e-01j,
        -8.5190e-01-5.2370e-01j, -2.0245e-01-9.7929e-01j,
         9.9928e-01+3.7908e-02j,  6.3533e-01+7.7224e-01j,
         5.0709e-01-8.6189e-01j, -9.7992e-01+1.9939e-01j,
         

In [62]:
f_phase = torch.diag(optimal_phase)
cap_g = 0
cap_f =0
dlinkse =0
randomse = 0
onephase=0
one = torch.diag(torch.ones(M,dtype=torch.cdouble)).to(device)
#optimal_P = torch.sqrt(optimal_P)
I = torch.eye(Nr**2,Nr**2).to(device)
print(optimal_P)
for i in range(len(optimal_P)):
    cap_g += torch.real(torch.log2(torch.det(I+(optimal_P[i]/P_noise)*(H2[i,:,:]@torch.diag(g_phase)@H1+H0[i,:,:])@((H2[i,:,:]@torch.diag(g_phase)@H1+H0[i,:,:]).conj().T))))
    cap_f += torch.real(torch.log2(torch.det(I+(optimal_P[i]/P_noise)*(H2[i,:,:]@f_phase@H1+H0[i,:,:])@((H2[i,:,:]@f_phase@H1+H0[i,:,:]).conj().T))))
    dlinkse += torch.real(torch.log2(torch.det(I+(optimal_P[i]/P_noise)*(H0[i,:,:])@(H0[i,:,:]).conj().T)))
    randomse += torch.real(torch.log2(torch.det(I+(optimal_P[i]/P_noise)*(H2[i,:,:]@rand_phi@H1+H0[i,:,:])@((H2[i,:,:]@rand_phi@H1+H0[i,:,:]).conj().T))))
    onephase += torch.real(torch.log2(torch.det(I+(optimal_P[i]/P_noise)*(H2[i,:,:]@one@H1+H0[i,:,:])@((H2[i,:,:]@one@H1+H0[i,:,:]).conj().T))))
print('gradient',cap_g.item())
print('Sequential',cap_f.item())
print('random',randomse.item())
print('direct only',dlinkse.item())
print('unit phase',onephase.item())


tensor([4.5386, 5.5163, 5.3926, 4.5524], device='cuda:0', dtype=torch.float64)
gradient 9.705969202254582
Sequential 10.335586439858162
random 9.625451021391983
direct only 4.536611533122023
unit phase 10.498286267940944


In [46]:
print(H22.shape)
print(torch.linalg.matrix_rank(H22))
print(H1_inv.shape)
print(torch.linalg.matrix_rank(H1_inv))
print(caschannel.shape)
print(torch.linalg.matrix_rank(caschannel))




print(optimal_phase.shape)

torch.Size([64, 128])
tensor(64, device='cuda:0')
torch.Size([16, 128])
tensor(3, device='cuda:0')
torch.Size([1024, 16384])
tensor(192, device='cuda:0')
torch.Size([64])
